<a href="https://colab.research.google.com/github/Nabin9909/mlpc/blob/main/Earthquake_Detection_ParallelML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Earthquake Detection using 1D CNN and Parallel Processing
This notebook demonstrates preprocessing, model training and evaluation using GPU and parallelism for real-time earthquake detection.

In [1]:
# Imports
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import time
from multiprocessing import Pool
import matplotlib.pyplot as plt

## Simulate Earthquake and Noise Data

In [2]:
def generate_waveform(is_quake, length=3000):
    x = np.linspace(0, 10*np.pi, length)
    signal = np.sin(x) * (np.random.rand(length) if is_quake else 0.1*np.random.rand(length))
    return signal + 0.2*np.random.randn(length)

X = np.array([generate_waveform(i%2==0) for i in range(1000)])
y = np.array([i%2 for i in range(1000)])

## Define and Train 1D CNN Model

In [3]:
X = X[..., np.newaxis]  # Add channel dimension
model = Sequential([
    Conv1D(16, 5, activation='relu', input_shape=(3000, 1)),
    MaxPooling1D(2),
    Conv1D(32, 5, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='loss', patience=3)

start = time.time()
model.fit(X, y, epochs=10, batch_size=32, callbacks=[early_stop], verbose=1)
end = time.time()
print(f"Training completed in {end - start:.2f} seconds")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 114ms/step - accuracy: 0.9231 - loss: 0.1468
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - accuracy: 1.0000 - loss: 7.6541e-06
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step - accuracy: 1.0000 - loss: 9.3617e-06
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 115ms/step - accuracy: 1.0000 - loss: 6.2190e-06
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step - accuracy: 1.0000 - loss: 6.3874e-06
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step - accuracy: 1.0000 - loss: 5.3875e-06
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step - accuracy: 1.0000 - loss: 9.3421e-06
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 79ms/step - accuracy: 1.0000 - loss: 1.7924e-06
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step - accuracy: 1.0000 - loss: 1.0422e-06
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 104ms/step - accuracy: 1.0000 - loss: 2.9412e-06
Training completed in 44.17 seconds


## Evaluate the Model

In [4]:
loss, acc = model.evaluate(X, y)
print(f"Accuracy: {acc*100:.2f}%")

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 1.0000 - loss: 4.2471e-08
Accuracy: 100.00%
